In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from pathlib import Path
import datetime

import dspy
from vf_musique.data import prepare_dataset
from vf_musique.metrics import exact_match, f1
from vf_musique.rewards import extract_all_retrieved_doc_ids

In [3]:
def setup_mlflow():
    import mlflow
    import mlflow.dspy

    mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))
    mlflow.set_experiment("dspy-gepa-musique")
    mlflow.dspy.autolog(
        log_compiles=True,
        log_evals=True,
        log_traces_from_compile=True,
    )
    print(f"✅ MLflow tracking enabled at {os.getenv('MLFLOW_TRACKING_URI')}")

setup_mlflow()

✅ MLflow tracking enabled at http://localhost:5005


In [4]:
EXP_ID = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
EXP_DIR = Path(f"../outputs/dspy/gepa-musique/{EXP_ID}")
EXP_DIR.mkdir(parents=True, exist_ok=True)
EXP_DIR

PosixPath('../outputs/dspy/gepa-musique/20250930_171202')

In [5]:
lm = dspy.LM(
    "openai/Qwen/Qwen3-8B",
    temperature=0.6,
    max_tokens=8192,
    api_key="local",
    api_base="http://0.0.0.0:8000/v1",
)
dspy.configure(lm=lm)

# reflection_lm = dspy.LM("gemini/gemini-2.5-pro", api_key=os.getenv("GEMINI_API_KEY"), max_tokens=16384,)
reflection_lm = dspy.LM(
    "openai/Qwen/Qwen3-32B",
    temperature=0.6,
    max_tokens=16384,
    api_key="local",
    api_base="http://0.0.0.0:8001/v1",
)

In [6]:
lm(messages=[{"role": "user", "content": "Hello"}])

['\n\nHello! 😊 How can I assist you today?']

Trace(trace_id=tr-c9e876584d7a4ecc176a0b8e234963ff)

In [7]:
reflection_lm(messages=[{"role": "user", "content": "What is largest prime number below 10?"}])

['\n\nThe largest prime number below 10 is **7**. \n\nTo determine this, we first identify all prime numbers less than 10. A prime number is a natural number greater than 1 that has no positive divisors other than 1 and itself. Evaluating the numbers from 2 to 9:\n\n- **2** is prime (divisible only by 1 and 2).\n- **3** is prime (divisible only by 1 and 3).\n- **4** is not prime (divisible by 2).\n- **5** is prime (divisible only by 1 and 5).\n- **6** is not prime (divisible by 2 and 3).\n- **7** is prime (divisible only by 1 and 7).\n- **8** is not prime (divisible by 2 and 4).\n- **9** is not prime (divisible by 3).\n\nAmong the prime numbers 2, 3, 5, and 7, the largest is **7**.\n\n---\n\n$$\n\\boxed{7}\n$$']

Trace(trace_id=tr-1bdedb47e72e097d70d93bb34d8904b7)

In [8]:
import random


def prepare_musique_dataset(datasets_str: str = "bdsaglam/musique,answerable,train", noise_rate: float = 1.0):
    """Load and prepare MuSiQue dataset using vf_musique data functions."""
    # Use the official vf_musique data preparation
    dataset = prepare_dataset(datasets_str, noise_rate=noise_rate)

    # Convert to DSPy examples
    processed_examples = []
    for x in dataset:
        # Get supporting document IDs
        supporting_doc_ids = [doc["id"] for doc in x["info"]["docs"] if doc.get("is_supporting")]

        # Create DSPy example
        example = dspy.Example(
            question=x["question"],
            answer=x["answer"],
            answers=x["info"]["answers"],  # All valid answer forms
            docs=x["info"]["docs"],  # All documents
            supporting_ids=supporting_doc_ids,  # IDs of supporting docs
            n_hops=x["info"]["n_hops"],  # Number of hops
        ).with_inputs("question", "docs")

        processed_examples.append(example)

    return processed_examples

ds = prepare_musique_dataset(datasets_str="bdsaglam/musique-mini,answerable,train", noise_rate=1.0)
random.Random(89).shuffle(ds)
train_size = int(len(ds)*0.60)
train_ds, val_ds = ds[:train_size], ds[train_size:]
test_ds = prepare_musique_dataset(datasets_str="bdsaglam/musique-mini,answerable,validation[:50]", noise_rate=1.0)

train_ds = train_ds[:30]
val_ds = val_ds[:30]

Map: 100%|##########| 300/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map: 100%|##########| 50/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [9]:
from pydantic import BaseModel
from agents import RunContextWrapper
from vf_musique.tools import make_retrieve_tool, ToolContext


class GenerateSearchQuery(dspy.Signature):
    """Given a multi-hop question and information collected so far, generate a search query
    to find the next piece of information needed to answer the question.
    Focus on entities, dates, or facts that need to be resolved step by step."""

    question: str = dspy.InputField(desc="The multi-hop question to answer")
    collected_info: str = dspy.InputField(desc="Information collected from previous retrieval steps")
    search_query: str = dspy.OutputField(desc="Search query for the next retrieval step")
    top_n: int = dspy.OutputField(desc="Number of documents to retrieve. 1 <= top_n <= 3")

class KeyInformation(BaseModel):
    info: str 
    source_doc_id: str

    def format(self):
        return f"{self.info}[{self.source_doc_id}]"

class ExtractInformation(dspy.Signature):
    """Given a question and retrieved documents, extract the key information
    that helps answer the question or leads to the next retrieval step.
    Focus on entities, relationships, dates, and facts."""

    question: str = dspy.InputField(desc="The multi-hop question to answer")
    documents: str = dspy.InputField(desc="Retrieved documents from search")
    key_informations: list[KeyInformation] = dspy.OutputField(desc="Key information(s) extracted from retrieved document(s)")

class DecideInfoCollection(dspy.Signature):
    question: str = dspy.InputField(desc="The multi-hop question to answer")
    all_information: str = dspy.InputField(desc="All information collected during retrieval")
    has_collected_enough_info: bool = dspy.OutputField(desc="Has enough information been collected to answer question?")

class GenerateAnswer(dspy.Signature):
    """Given a multi-hop question and all collected information, provide a concise answer.
    The answer should directly address what the question asks for.
    Be specific and use the exact entities/dates/facts from the documents."""

    question: str = dspy.InputField(desc="The multi-hop question to answer")
    all_information: str = dspy.InputField(desc="All information collected during retrieval")
    answer: str = dspy.OutputField(desc="Final answer to the question")
    citations: list[str] = dspy.OutputField(desc="List of document IDs cited for the answer, e.g. `[4,9]`")


class MultiHopQA(dspy.Module):
    """Multi-hop question answering module for MuSiQue."""

    def __init__(self, retriever_name: str = "hybrid", max_iter: int = 10):
        self.retriever_name = retriever_name
        self.max_iter = 10

        # Create the retrieve tool
        self.retrieve_tool = make_retrieve_tool(retriever_name, default_top_n=2)

        # Create modules with typed signatures
        self.generate_query = dspy.ChainOfThought(GenerateSearchQuery)
        self.extract_info = dspy.ChainOfThought(ExtractInformation)
        self.decide_info_collect = dspy.ChainOfThought(DecideInfoCollection)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question: str, docs: list, **kwargs) -> dspy.Prediction:
        """
        Forward pass for multi-hop QA.

        Args:
            question: The multi-hop question to answer
            docs: List of documents available for retrieval
        """
        collected_info = []
        retrieved_doc_ids = []

        # Create a context object that mimics the verifiers tool environment
        run_context_wrapper = RunContextWrapper[ToolContext](context=ToolContext(info=dict(docs=docs)))

        for hop_idx in range(self.max_iter):
            # Generate search query
            if hop_idx == 0:
                # First hop: use the original question
                query = question
                top_n = 2
            else:
                # Subsequent hops: generate query based on collected info
                query_pred = self.generate_query(
                    question=question,
                    collected_info="\n".join([item.format() for item in collected_info]) if collected_info else "No information collected yet",
                )
                query = query_pred.search_query
                top_n = max(min(query_pred.top_n, 3), 1)

            # Retrieve documents using the MuSiQue retrieve tool
            retrieved_text = self.retrieve_tool(run_context_wrapper, query=query, top_n=top_n)

            # Extract document IDs from retrieved text using the official function
            doc_ids = extract_all_retrieved_doc_ids(retrieved_text)
            for doc_id in doc_ids:
                if doc_id not in retrieved_doc_ids:
                    retrieved_doc_ids.append(doc_id)

            # Extract key information from retrieved documents
            info_pred = self.extract_info(question=question, documents=retrieved_text)
            collected_info.extend(info_pred.key_informations)

            decision_pred = self.decide_info_collect(question=question, all_information="\n".join([item.format() for item in collected_info]))        
            if decision_pred.has_collected_enough_info:
                break

        # Generate final answer based on all collected information
        answer_pred: GenerateAnswer = self.generate_answer(question=question, all_information="\n".join([item.format() for item in collected_info]))

        return dspy.Prediction(
            answer=answer_pred.answer,
            collected_info=collected_info,
            retrieved_doc_ids=retrieved_doc_ids,
            citations=answer_pred.citations,
        )

program = MultiHopQA()

In [10]:
example = train_ds[3]
example

Example({'question': 'The mosaic in the church in the city where Maria Tsiartsiani was born, is known as what?', 'answer': "Christ in majesty (or Ezekiel's Vision)", 'answers': ["Christ in majesty (or Ezekiel's Vision)", "christ in majesty (or ezekiel's vision)"], 'docs': [{'body': 'In the Iconoclastic era, figural mosaics were also condemned as idolatry. The Iconoclastic churches were embellished with plain gold mosaics with only one great cross in the apse like the Hagia Irene in Constantinople (after 740). There were similar crosses in the apses of the Hagia Sophia Church in Thessaloniki and in the Church of the Dormition in Nicaea. The crosses were substituted with the image of the Theotokos in both churches after the victory of the Iconodules (787–797 and in 8th–9th centuries respectively, the Dormition church was totally destroyed in 1922).', 'id': '0', 'is_supporting': False, 'text': '# Mosaic\nIn the Iconoclastic era, figural mosaics were also condemned as idolatry. The Iconocl

In [11]:
pred = program(example.question, example.docs)
pred

Prediction(
    answer="Christ in majesty (or Ezekiel's Vision)",
    collected_info=[KeyInformation(info='Maria Tsiartsiani was born in Thessaloniki', source_doc_id='1'), KeyInformation(info="The mosaic in the Church of Hosios David in Thessaloniki is known as the 'Christ in majesty (or Ezekiel's Vision)' mosaic.", source_doc_id='5')],
    retrieved_doc_ids=['1', '2', '12', '5'],
    citations=['5']
)

Trace(trace_id=tr-465a91fe2dfac98ba44f523ed36f2b3a)

In [12]:
def metric_retrieval_recall(example, pred, trace=None):
    """Retrieval recall metric - fraction of supporting documents found."""
    if not example.supporting_ids:
        return 1.0  # No supporting documents to evaluate

    gold_ids = set(example.supporting_ids)
    retrieved_ids = set(pred.retrieved_doc_ids)

    if not gold_ids:
        return 1.0

    found = gold_ids.intersection(retrieved_ids)
    return len(found) / len(gold_ids)


def metric_retrieval_precision(example, pred, trace=None):
    """Retrieval precision metric - fraction of retrieved documents that are supporting."""
    if not example.supporting_ids:
        return 1.0

    gold_ids = set(example.supporting_ids)
    retrieved_ids = set(pred.retrieved_doc_ids)
    found = gold_ids.intersection(retrieved_ids)
    return len(found) / len(retrieved_ids)


def metric_answer_exact_match(example, pred, trace=None):
    """Exact match metric for MuSiQue using the official metrics."""
    return exact_match(pred.answer, example.answers)


def metric_answer_f1_score(example, pred, trace=None):
    """Token-level F1 score using the official metrics."""
    return f1(pred.answer, example.answers)


def metric_citation_f1(example, pred, trace=None):
    """Citation accuracy metrics - precision, recall, F1 for cited document IDs."""
    # Convert to sets for easy comparison
    gold_ids = set(example.supporting_ids) if example.supporting_ids else set()
    cited_ids = set(str(doc_id) for doc_id in pred.citations)  # Ensure string format

    # Handle edge cases
    if not gold_ids:
        raise ValueError("Supporting docs must be provided for citation metric")

    if not cited_ids:
        # No citations given but some needed
        return 0.0

    # Calculate standard precision/recall/F1
    correct_citations = cited_ids & gold_ids
    precision = len(correct_citations) / len(cited_ids)
    recall = len(correct_citations) / len(gold_ids)
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return f1


def metric(example, pred, trace=None):
    """Combined metric for MuSiQue: weighted by number of hops."""
    retrieval_recall_score = metric_retrieval_recall(example, pred, trace)
    retrieval_precision_score = metric_retrieval_precision(example, pred, trace)
    answer_f1_score = metric_answer_f1_score(example, pred, trace)
    citation_f1 = metric_citation_f1(example, pred, trace)

    # Combine metrics: EM and F1 for answer quality, retrieval recall for completeness,
    # citation F1 for proper attribution
    score_weight_pairs = [
        (retrieval_recall_score, 0.9),  # Retrieval recall for finding supporting docs
        (retrieval_precision_score, 0.5),  # Retrieval precision for finding supporting docs
        (answer_f1_score, 1.0),  # F1
        (citation_f1, 0.7),  # Citation accuracy for proper attribution
    ]

    return sum(score * weight for score, weight in score_weight_pairs) / sum(weight for _, weight in score_weight_pairs)

In [13]:
metric(example, pred)

0.8440860215053764

In [14]:
# Evaluate original program
print("📊 Evaluating ORIGINAL program...")
original_evaluate = dspy.Evaluate(
    devset=test_ds,
    metric=metric,
    num_threads=8,
    display_table=False,
    display_progress=True
)
original_eval_result = original_evaluate(program)

📊 Evaluating ORIGINAL program...
Average Metric: 34.81 / 50 (69.6%): : 51it [12:36, 14.84s/it]                                                                                                                                         

2025/09/30 17:26:37 INFO dspy.evaluate.evaluate: Average Metric: 34.80956083099297 / 50 (69.6%)



🏃 View run eval at: http://localhost:5005/#/experiments/1/runs/4b8d61076ae649a09876c1bb09cf8a32
🧪 View experiment at: http://localhost:5005/#/experiments/1


[Trace(trace_id=tr-8e36050c243e957a3a1fc4068fe8aa49), Trace(trace_id=tr-203927f7371d90f4111bc93d66db35f3), Trace(trace_id=tr-752125dce1f42af2f71da532346d7eb2), Trace(trace_id=tr-3a7508a6f8678c6895c0e965e2e3e4ad), Trace(trace_id=tr-81309312aeafdac55c36d4ec8fdd89ed), Trace(trace_id=tr-ba4521940eeb57d908c62ad7a8da6dbd), Trace(trace_id=tr-05dbaa6327e42661ca39b1c57eb44424), Trace(trace_id=tr-33813726f4ef0ac67386927cd5e24199), Trace(trace_id=tr-15da6431d483c8e2fa08cf70bbbf041f), Trace(trace_id=tr-348c90af0804ef5445be1023b9284843)]

## GEPA Optimization

GEPA is a reflective prompt optimizer that uses textual feedback to improve performance. We'll create feedback functions for each evaluation aspect and optimize our multi-hop QA program.


In [15]:
def feedback_retrieval_recall(example, pred):
    """Generate feedback for retrieval recall evaluation."""
    gold_ids = set(example.supporting_ids)
    retrieved_ids = set(pred.retrieved_doc_ids)
    found = gold_ids.intersection(retrieved_ids)
    recall_score = len(found) / len(gold_ids)

    if recall_score == 1.0:
        feedback = f"Perfect retrieval! You found all {len(gold_ids)} supporting documents: {sorted(found)}"
    elif recall_score >= 0.5:
        missing_ids = gold_ids - found
        feedback = (
            f"Good retrieval (recall: {recall_score:.2f}). Found {len(found)} out of {len(gold_ids)} "
            f"supporting documents. Missing: {sorted(missing_ids)}. Consider refining your search queries "
            f"to find the remaining relevant documents."
        )
    else:
        missing_ids = gold_ids - found
        feedback = (
            f"Poor retrieval (recall: {recall_score:.2f}). Only found {len(found)} out of {len(gold_ids)} "
            f"supporting documents. Missing critical documents: {sorted(missing_ids)}. "
            f"Your search queries need to be more comprehensive and targeted."
        )

    return recall_score, feedback


def feedback_retrieval_precision(example, pred):
    """Generate feedback for retrieval precision evaluation."""
    gold_ids = set(example.supporting_ids)
    retrieved_ids = set(pred.retrieved_doc_ids)

    if not retrieved_ids:
        return 0.0, "No documents were retrieved. Your search queries need to find relevant documents."

    found = gold_ids.intersection(retrieved_ids)
    precision_score = len(found) / len(retrieved_ids)
    irrelevant_docs = retrieved_ids - gold_ids

    if precision_score == 1.0:
        feedback = (
            f"Perfect precision! All {len(retrieved_ids)} retrieved documents are supporting documents: {sorted(found)}"
        )
    elif precision_score >= 0.7:
        feedback = (
            f"Good precision (precision: {precision_score:.2f}). {len(found)} out of {len(retrieved_ids)} "
            f"retrieved documents are relevant. Irrelevant docs: {sorted(irrelevant_docs)}. "
            f"Consider making your search queries more specific to avoid irrelevant documents."
        )
    elif precision_score >= 0.3:
        feedback = (
            f"Moderate precision (precision: {precision_score:.2f}). Only {len(found)} out of {len(retrieved_ids)} "
            f"retrieved documents are relevant. Many irrelevant docs retrieved: {sorted(irrelevant_docs)}. "
            f"Your search queries are too broad - focus on more specific terms and entities."
        )
    else:
        feedback = (
            f"Poor precision (precision: {precision_score:.2f}). Only {len(found)} out of {len(retrieved_ids)} "
            f"retrieved documents are relevant. Most retrieved docs are irrelevant: {sorted(irrelevant_docs)}. "
            f"Your search queries are retrieving too many irrelevant documents. Be much more specific and targeted."
        )

    return precision_score, feedback


def feedback_answer_exact_match(example, pred):
    """Generate feedback for exact match evaluation."""
    em_score = exact_match(pred.answer, example.answers)

    if em_score == 1.0:
        feedback = f"Perfect! You provided the exact correct answer: '{pred.answer}'. This matches the expected answer exactly."
    else:
        # Find the best matching answer for more specific feedback
        best_answer = example.answers[0] if example.answers else "N/A"
        feedback = (
            f"Your answer '{pred.answer}' doesn't exactly match the expected answer '{best_answer}'. "
            f"Consider being more precise with entity names, dates, and specific facts."
        )

    return em_score, feedback


def feedback_answer_f1_score(example, pred):
    """Generate feedback for F1 score evaluation."""
    f1_score = f1(pred.answer, example.answers)

    if f1_score >= 0.9:
        feedback = f"Excellent! Your answer has high overlap (F1: {f1_score:.2f}) with the expected answer. Good token-level accuracy."
    elif f1_score >= 0.5:
        feedback = (
            f"Good partial match (F1: {f1_score:.2f}). Your answer contains relevant information but "
            f"could be more complete or precise. Consider including more specific details from the retrieved documents."
        )
    else:
        best_answer = example.answers[0] if example.answers else "N/A"
        feedback = (
            f"Low overlap (F1: {f1_score:.2f}) with expected answer. Your answer '{pred.answer}' "
            f"differs significantly from '{best_answer}'. Focus on extracting the specific information "
            f"requested in the question."
        )

    return f1_score, feedback


def feedback_citation_f1(example, pred):
    """Generate feedback for citation F1 evaluation."""
    gold_ids = set(example.supporting_ids) if example.supporting_ids else set()
    cited_ids = set(str(doc_id) for doc_id in pred.citations)

    if not gold_ids:
        return 1.0, "No supporting documents to cite."

    if not cited_ids:
        feedback = f"You didn't cite any documents, but should have cited: {sorted(gold_ids)}. Always cite the documents that support your answer."
        return 0.0, feedback

    correct_citations = cited_ids & gold_ids
    precision = len(correct_citations) / len(cited_ids)
    recall = len(correct_citations) / len(gold_ids)
    citation_f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    if citation_f1 >= 0.9:
        feedback = f"Excellent citations (F1: {citation_f1:.2f})! You properly cited the supporting documents: {sorted(correct_citations)}"
    elif citation_f1 >= 0.5:
        incorrect_citations = cited_ids - gold_ids
        missing_citations = gold_ids - cited_ids
        feedback = f"Good citations (F1: {citation_f1:.2f}). Correct: {sorted(correct_citations)}. "
        if incorrect_citations:
            feedback += f"Unnecessary: {sorted(incorrect_citations)}. "
        if missing_citations:
            feedback += f"Missing: {sorted(missing_citations)}. "
        feedback += "Be more precise about which documents actually support your answer."
    else:
        incorrect_citations = cited_ids - gold_ids
        missing_citations = gold_ids - cited_ids
        feedback = (
            f"Poor citations (F1: {citation_f1:.2f}). You cited {sorted(cited_ids)} but should cite {sorted(gold_ids)}. "
            f"Focus on identifying which documents directly support your answer claims."
        )

    return citation_f1, feedback


def metric_with_feedback(example, pred, trace=None, pred_name=None, pred_trace=None):
    """
    Combined metric for MuSiQue with feedback for GEPA optimization.
    Returns a dspy.Prediction with score (float) and feedback (str).

    The feedback is targeted at specific predictors when pred_name is provided,
    helping GEPA understand how to improve each component.
    """
    # Compute feedback and scores for all metrics
    score_answer_f1, fb_answer_f1 = feedback_answer_f1_score(example, pred)
    score_retrieval_recall, fb_retrieval_recall = feedback_retrieval_recall(example, pred)
    score_retrieval_precision, fb_retrieval_precision = feedback_retrieval_precision(example, pred)
    score_citation_f1, fb_citation_f1 = feedback_citation_f1(example, pred)

    # Combined score: weighted average of all metrics (same as original metric)
    score_weight_pairs = [
        (score_answer_f1, 1.0),  # Answer F1
        (score_retrieval_recall, 0.9),  # Retrieval recall for finding supporting docs
        (score_retrieval_precision, 0.5),  # Retrieval precision for finding supporting docs
        (score_citation_f1, 0.7),  # Citation accuracy for proper attribution
    ]

    total_score = sum(score * weight for score, weight in score_weight_pairs) / sum(
        weight for _, weight in score_weight_pairs
    )

    # Provide targeted feedback based on the predictor being optimized
    if pred_name == "generate_query.predict":
        # Focus on query generation quality and retrieval effectiveness
        feedback = (
            fb_retrieval_recall
            + " "
            + fb_retrieval_precision
            + " "
            + "Your search queries should be both comprehensive (high recall) and specific (high precision). "
            "Consider what entities, relationships, or facts are needed for each hop of reasoning."
        )

    elif pred_name == "extract_info.predict":
        # Focus on information extraction quality
        feedback = (
            fb_answer_f1
            + " "
            + (
                "Focus on extracting the most relevant facts, entities, and relationships from the retrieved documents. "
                "Make sure to capture information that directly helps answer the question or leads to the next reasoning step."
            )
        )

    elif pred_name == "generate_answer.predict":
        # Focus on answer generation and citation quality
        feedback = (
            fb_answer_f1
            + " "
            + fb_citation_f1
            + " "
            + (
                "Provide precise, complete answers using the exact information from the retrieved documents. "
                "Always cite the document IDs that support your answer claims."
            )
        )
    else:
        # Generic feedback combining all aspects
        feedback = "\n".join([
            "Overall performance breakdown:",
            f"- Answer F1 Score: {fb_answer_f1}",
            f"- Retrieval Recall: {fb_retrieval_recall}",
            f"- Retrieval Precision: {fb_retrieval_precision}",
            f"- Citations F1 Score: {fb_citation_f1}"
        ])

    return dspy.Prediction(score=total_score, feedback=feedback)


In [16]:
# Test the feedback metric on our example
feedback_result = metric_with_feedback(example, pred)
print(f"Score: {feedback_result.score:.3f}")
print(f"Feedback: {feedback_result.feedback}")


Score: 0.844
Feedback: Overall performance breakdown:
- Answer F1 Score: Excellent! Your answer has high overlap (F1: 1.00) with the expected answer. Good token-level accuracy.
- Retrieval Recall: Perfect retrieval! You found all 2 supporting documents: ['1', '5']
- Retrieval Precision: Moderate precision (precision: 0.50). Only 2 out of 4 retrieved documents are relevant. Many irrelevant docs retrieved: ['12', '2']. Your search queries are too broad - focus on more specific terms and entities.
- Citations F1 Score: Good citations (F1: 0.67). Correct: ['5']. Missing: ['1']. Be more precise about which documents actually support your answer.


In [17]:
from dspy import GEPA

# Set up GEPA optimizer with reflection LM for optimization
optimizer = GEPA(
    metric=metric_with_feedback,
    auto="light",  # Use light budget for faster experimentation. Use "heavy" for best performance
    num_threads=8,
    track_stats=True,
    use_merge=False,
    reflection_lm=reflection_lm  
)

print("✅ GEPA optimizer configured")

✅ GEPA optimizer configured


In [ ]:
# Run GEPA optimization
print("🚀 Starting GEPA optimization...")

optimized_program = optimizer.compile(
    program,
    trainset=train_ds,
    valset=val_ds,
)

print("✅ GEPA optimization completed!")


2025/09/30 17:26:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '72261987ca3342a2a49dd381825e0dce', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current dspy workflow
2025/09/30 17:26:38 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 1765 metric calls of the program. This amounts to 29.42 full evals on the train+val set.
2025/09/30 17:26:38 INFO dspy.teleprompt.gepa.gepa: Using 30 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.


🚀 Starting GEPA optimization...


GEPA Optimization:   0%|                                                                                                                                               | 0/1765 [00:00<?, ?rollouts/s]2025/09/30 17:42:02 INFO dspy.evaluate.evaluate: Average Metric: 17.871924849344207 / 30 (59.6%)


🏃 View run eval_0 at: http://localhost:5005/#/experiments/1/runs/73771306fb7240a7a3f0a3338e436b5f
🧪 View experiment at: http://localhost:5005/#/experiments/1


2025/09/30 17:42:02 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.5957308283114735
GEPA Optimization:   2%|██▏                                                                                                                                | 30/1765 [15:24<14:51:08, 30.82s/rollouts]2025/09/30 17:42:02 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.5957308283114735


...

2025/09/30 21:53:21 INFO dspy.teleprompt.gepa.gepa: Iteration 21: New subsample score is not better, skipping
GEPA Optimization:  22%|████████████████████████████▋                                                                                                   | 396/1765 [4:26:43<15:39:17, 41.17s/rollouts]2025/09/30 21:53:21 INFO dspy.teleprompt.gepa.gepa: Iteration 22: Selected program 6 score: 0.5758294930875576


Average Metric: 2.16 / 3 (72.0%): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:59<00:00, 39.96s/it]

2025/09/30 21:55:22 INFO dspy.evaluate.evaluate: Average Metric: 2.1586021505376345 / 3 (72.0%)



🏃 View run eval_51 at: http://localhost:5005/#/experiments/1/runs/89fc9e0ff9ac49ab955ff92aad73d84d
🧪 View experiment at: http://localhost:5005/#/experiments/1


2025/09/30 21:56:10 INFO dspy.teleprompt.gepa.gepa: Iteration 22: Proposed new text for decide_info_collect.predict: You are tasked with determining whether the provided `all_information` contains sufficient and directly relevant data to answer the given `question`. Your output must include two components:  

1. **Reasoning**:  
   - Identify the key entities, relationships, and temporal/geographic links in the question (e.g., "city where RSA Conference was held" → "San Francisco").  
   - Map these entities to the `all_information` to verify if **all required data points** (e.g., dates, locations, historical events) are explicitly stated.  
   - Check if **supporting documents** (e.g., [11], [13]) directly address the question’s components. Avoid citing irrelevant documents.  
   - If the information is insufficient, explain which critical details are missing (e.g., "Ladakh's integration into Qing China" in Example 2).  

2. **has_collected_enough_info**:  
   - Output `True` **only i

In [ ]:
optimized_program.save(str(EXP_DIR / "optimized-program"), save_program=True)

### Examine Optimized Prompts

Let's look at how GEPA improved the prompts for each predictor:


In [ ]:
for name, pred in optimized_program.named_predictors():
    print("=" * 60)
    print(f"Predictor: {name}")
    print("=" * 60)
    print("Optimized Instructions:")
    print(pred.signature.instructions)
    print("*" * 60)


### Evaluate Optimized Program

Compare the performance before and after GEPA optimization:


In [ ]:
print("\\n📊 Evaluating OPTIMIZED program...")
# Evaluate optimized program  
optimized_evaluate = dspy.Evaluate(
    devset=test_ds,
    metric=metric,
    num_threads=8,
    display_table=False,
    display_progress=True
)
optimized_eval_result = optimized_evaluate(optimized_program)

In [ ]:
print("\\n" + "=" * 50)
print("🏆 PERFORMANCE COMPARISON")
print("=" * 50)
print(f"Original Program Score:  {original_eval_result.score:.3f}")
print(f"Optimized Program Score: {optimized_eval_result.score:.3f}")
print(f"Improvement:            {optimized_eval_result.score - original_eval_result.score:+.3f}")
print(f"Relative Improvement:   {((optimized_eval_result.score / original_eval_result.score) - 1) * 100:+.1f}%")

### GEPA Optimization Analysis

Analyze the detailed optimization results:


In [ ]:
# Analyze GEPA optimization trajectory
if hasattr(optimized_program, 'detailed_results'):
    results = optimized_program.detailed_results
    
    print("🔍 GEPA Optimization Details:")
    print(f"- Total candidates explored: {len(results.candidates)}")
    print(f"- Best candidate index: {results.best_idx}")
    print(f"- Best validation score: {results.val_aggregate_scores[results.best_idx]:.3f}")
    print(f"- Discovery evaluations used: {sum(results.discovery_eval_counts)}")
    
    # Show score progression
    print("\\n📈 Score progression:")
    for i, score in enumerate(results.val_aggregate_scores[:10]):  # Show first 10
        print(f"Candidate {i}: {score:.3f}")
    
    if len(results.val_aggregate_scores) > 10:
        print(f"... and {len(results.val_aggregate_scores) - 10} more candidates")
else:
    print("Detailed results not available (set track_stats=True in GEPA constructor)")


In [ ]:
# Test optimized program on the same example
example = test_ds[0]

print("🧪 Testing optimized program on example:")
print(f"Question: {example.question}")
print(f"Expected Answer: {example.answer}")
print(f"Supporting Docs: {example.supporting_ids}")
print()

pred = program(example.question, example.docs)
optimized_pred = optimized_program(example.question, example.docs)

print("📋 ORIGINAL vs OPTIMIZED Results:")
print("-" * 50)
print("ORIGINAL:")
print(f"  Answer: {pred.answer}")
print(f"  Retrieved docs: {pred.retrieved_doc_ids}")
print(f"  Cited docs: {pred.citations}")

print("OPTIMIZED:")
print(f"  Answer: {optimized_pred.answer}")
print(f"  Retrieved docs: {optimized_pred.retrieved_doc_ids}")
print(f"  Cited docs: {optimized_pred.citations}")

print("🎯 Metric Comparison:")
original_metric_result = metric_with_feedback(example, pred)
optimized_metric_result = metric_with_feedback(example, optimized_pred)
print(f"Original score: {original_metric_result.score:.3f}")
print(f"Original feedback: {original_metric_result.feedback}")
print()
print(f"Optimized score: {optimized_metric_result.score:.3f}")
print(f"Optimized feedback: {optimized_metric_result.feedback}")


Can we measure instruction quality by using them with a larger model to see if it gets questions right?